<a href="https://colab.research.google.com/github/ancestor9/24_fall_textmining_NLP/blob/main/%EA%B3%BC%EC%A0%9C%EC%A0%9C%EC%B6%9C_1119_LDA_practice_fetchnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import random
import matplotlib.pyplot as plt
!pip install pyLDAvis --quiet
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models
import gensim
import gensim.corpora as corpora
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords

# NLTK 다운로드 (최초 실행 시에만 필요)
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# prompt: fetch_20newsgroups의 category를 모두 보려면

from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='train')
newsgroups.target_names

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [1]:
#newsgroups.keys()

In [ ]:
target_names = newsgroups.target_names

topics = np.random.randint(3, 8)
categories = np.random.choice(target_names, topics) # 3개 ~ 8개 주제 분류 선택

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 1. 뉴스 데이터 수집 및 데이터프레임 생성
newsgroups = fetch_20newsgroups(subset='train',
                                categories=categories,
                                remove=('headers', 'footers', 'quotes'))

# 데이터프레임으로 변환
df = pd.DataFrame({'text': newsgroups.data,
                #    'category': newsgroups.target
                   }
                  )
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text
0,\nI know. You have this fucked up idea that a...
1,\n\nYou may have to define your serial ports u...
2,"\nSnort. Ah, there go my sinuses again.\n\n\n..."
3,I am getting Garbled output when serial printi...
4,Can anyone tell me if a bloodcount of 40 when ...
...,...
2370,\nWhen the Lunar Society's $500M estimate of t...
2371,"I missed the original post, but aren't the Exp..."
2372,A relative of mine was recently diagnosed with...
2373,\n\nCould you use some sort of mechanical ches...


### <font color='orange'>**LDA분석을 통해 Topic의 개수와 주제의 이름을 추론하고 그 이유를 설명하시오.**
#### 1. Topic 모델 html파일 제출
#### 2. Topic 게수, 이름과 그 판단 근거